In [1]:
import requests as req
from bs4 import BeautifulSoup

In [32]:
# --Constants-- #
base_url = 'https://www.nytimes.com'
week_url = '/books/best-sellers'
weeks_num = 104
cat_length = 5


# --Start Scraping-- #
best_seller = []
for week in range(1, weeks_num+1):
    week_dict = {
        'week': week,
        'data': list,
        }
    
    # --Fetch Data-- #
    try:
        
        page = req.get(base_url + week_url)
        if page.status_code == 200:
            soup = BeautifulSoup(page.text, 'html.parser')
            text_res = soup.find_all('div', class_='css-1le66x')
            img_url_res = soup.find_all('div', class_='css-9fprjv')
            categories = [cat.text for cat in soup.find_all('a', class_='css-nzgijy')]
        else:
            break
            print(f'Request status code {page.status_code}')
            
    except Exception as error:
        print('Problem fetching data, Err:', error)
    
    
    # --Create Generators-- #
    def text_gen(text):
        yield from text

    def img_gen(imgs):
        yield from imgs

    def cat_gen(category_list):
        yield from category_list

    if text_res and img_url_res and categories:
        text = text_gen(text_res)
        img_url = img_gen(img_url_res)
        category = cat_gen(categories)
    else:
        break
        print('One or more data lists is empty or missing')
    
    
    # --Save to Data Structure-- #
    data = []
    for cat in range(len(categories)):
        cat_dict = {
            'category': next(category),
            'books': list,
            }
        
        books = []
        position = 1
        for book in range(cat_length):
            book_dict = {
                'title': str,
                'position': int,
                'descreption': str,
                'image_url' : str,
                }
            
            info = next(text)
            img = next(img_url)
            
            book_dict['position'] = position
            book_dict['title'] = info.find('h3', class_="css-i1z3c1").text
            book_dict['descreption'] = info.find('p', class_='css-5yxv3r').text
            if book_dict.get('title') in img.find(class_='css-35otwa')['alt']:  # Making sure the book matches the img
                book_dict['image_url'] = img.find(class_='css-35otwa')['src']

            books.append(book_dict)
            position += 1
            
        cat_dict['books'] = books
        data.append(cat_dict)
    
    week_dict['data'] = data
    best_seller.append(week_dict)
    
    # --Assign Previous Week to URL request-- #
    prev_week = soup.find_all('div', class_='css-iouqpc')[0].find('a')
    week_url = prev_week['href']


In [34]:
# --Pretty Print Data-- #
best_seller